In [52]:
import json

tweets_data = []
tweets_file = open('geotagged_tweets_20160812-0912.jsons', "r")
print(tweets_file)

counter = 0
for line in tweets_file:
    counter = counter + 1
    if counter > 30000:
        break
    try:
        tweet = json.loads(line)
        tweets_data.append(tweet)
    except:
        continue
print(tweets_data[1:10])

<_io.TextIOWrapper name='geotagged_tweets_20160812-0912.jsons' mode='r' encoding='cp1252'>
[{'created_at': 'Fri Aug 12 10:04:02 +0000 2016', 'id': 764039733076897792, 'id_str': '764039733076897792', 'text': '@BarackObama \n@FBI\n@LORETTALYNCH \nALL IN COLLUSION TOGETHER \n\n#NOJUSTICE \n\n@realDonaldTrump \n#TrumpPence \n\nhttps://t.co/5GMNZq40V3', 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': 813286, 'in_reply_to_user_id_str': '813286', 'in_reply_to_screen_name': 'BarackObama', 'user': {'id': 82496193, 'id_str': '82496193', 'name': 'Red Octopus', 'screen_name': 'redoctapus', 'location': 'Ogletown, DE', 'url': None, 'description': 'AWSCWI Pipefitter USAF NRA I remember USA, Hunting Fishing, HarleyDavidsons- a time when we HS students could carry guns in our trks 2 school', 'protected': False, 'verified': False, 'followers_count

In [53]:
import os
import json
import numpy as np
import pandas as pd
import nltk
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import rgb2hex
from descartes import PolygonPatch
from shapely.geometry import Polygon, MultiPolygon
import plotly.offline as py#visualization
import plotly.graph_objs as go#visualization
import plotly.tools as tls#visualization
import plotly.figure_factory as ff#visualization


In [54]:
import seaborn as sns
import sklearn

In [55]:
from pandas.io.json import json_normalize
# Thank gawd for this package - all json fields straight into columns in a dataframe, no bullshit
tweets_df = json_normalize(tweets_data)

tweets_df.head()

,contributors,coordinates,coordinates.coordinates,coordinates.type,created_at,entities.hashtags,entities.media,entities.symbols,entities.urls,entities.user_mentions,...,user.profile_sidebar_fill_color,user.profile_text_color,user.profile_use_background_image,user.protected,user.screen_name,user.statuses_count,user.time_zone,user.url,user.utc_offset,user.verified
0,None,NaN,NaN,NaN,Fri Aug 12 10:04:00 +0000 2016,[],"[{'id': 764039718237409281, 'id_str': '7640397...",[],[],"[{'screen_name': 'theblaze', 'name': 'TheBlaze...",...,EFEFEF,333333,True,False,DUPUY77,17620,None,http://ggm-dupuy.com,NaN,False
1,None,NaN,NaN,NaN,Fri Aug 12 10:04:02 +0000 2016,"[{'text': 'NOJUSTICE', 'indices': [62, 72]}, {...",NaN,[],"[{'url': 'https://t.co/5GMNZq40V3', 'expanded_...","[{'screen_name': 'BarackObama', 'name': 'Barac...",...,000000,000000,False,False,redoctapus,5046,Eastern Time (US & Canada),None,-14400.0,False
2,None,NaN,NaN,NaN,Fri Aug 12 10:04:10 +0000 2016,[],"[{'id': 764039762973896704, 'id_str': '7640397...",[],[],"[{'screen_name': 'theblaze', 'name': 'TheBlaze...",...,EFEFEF,333333,True,False,DUPUY77,17621,None,http://ggm-dupuy.com,NaN,False
3,None,NaN,NaN,NaN,Fri Aug 12 10:04:21 +0000 2016,[],NaN,[],[],"[{'screen_name': 'HillaryClinton', 'name': 'Hi...",...,99CC33,3E4415,True,False,jjorgi30,2231,Melbourne,None,36000.0,False
4,None,NaN,NaN,NaN,Fri Aug 12 10:04:30 +0000 2016,"[{'text': 'CNN', 'indices': [0, 4]}, {'text': ...",NaN,[],[],[],...,F3F3F3,333333,True,False,bspence5,92605,None,None,NaN,False


In [74]:
cols = []

for col in tweets_df.columns:
    if col in ('created_at', 'place.country','place.country_code','place.name','is_quote_status','lang','text','user.created_at','user.description','user.name','user.location','place.place_type', 'user-screen_name', 'in_reply_to_screen_name','geo.type','geo.coordinates', 'retweeted_status-text'):
        cols.append(col)

neat_df = tweets_df.loc[:, cols]
neat_df.head()

,created_at,geo.coordinates,geo.type,in_reply_to_screen_name,is_quote_status,lang,place.country,place.country_code,place.name,place.place_type,text,user.created_at,user.description,user.location,user.name
0,Fri Aug 12 10:04:00 +0000 2016,NaN,NaN,theblaze,False,und,United States,US,Frontenac,city,@theblaze @realDonaldTrump https://t.co/TY9DlZ...,Fri Sep 02 14:54:17 +0000 2011,"Fashion photographer, love action and adventur...",Miami,GIL DUPUY
1,Fri Aug 12 10:04:02 +0000 2016,NaN,NaN,BarackObama,False,en,United States,US,Baton Rouge,city,@BarackObama \n@FBI\n@LORETTALYNCH \nALL IN CO...,Thu Oct 15 00:28:04 +0000 2009,"AWSCWI Pipefitter USAF NRA I remember USA, Hun...","Ogletown, DE",Red Octopus
2,Fri Aug 12 10:04:10 +0000 2016,NaN,NaN,theblaze,False,und,United States,US,Frontenac,city,@theblaze @realDonaldTrump https://t.co/n050DB...,Fri Sep 02 14:54:17 +0000 2011,"Fashion photographer, love action and adventur...",Miami,GIL DUPUY
3,Fri Aug 12 10:04:21 +0000 2016,NaN,NaN,HillaryClinton,False,en,Australia,AU,Melbourne,city,@HillaryClinton he will do in one year all the...,Tue Jun 02 01:59:59 +0000 2009,The best movies are written and directed by pe...,Australia,JanJorgenson
4,Fri Aug 12 10:04:30 +0000 2016,NaN,NaN,None,False,en,United States,US,Baltimore,city,#CNN #newday clear #Trump deliberately throwin...,Mon Aug 20 09:43:48 +0000 2012,"Retired, political wonk mother of one grandmot...",None,Beverly Spence


In [1]:
import pandas as pd
neat_df = pd.read_parquet('file.parquet')

In [2]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hessel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import re
#probably gotta clean it more but this is a first step
def clean_text(dataframe, col, bigrams = False):
    # make new column for accounts
    dataframe.loc[:, "accounts"] = dataframe[col].str.findall(r'@.*?(?=\s|$)')
    # make new column for hashtag
    dataframe.loc[:, "hashtag"] = dataframe[col].str.findall(r'#.*?(?=\s|$)')
    #remove links
    dataframe.loc[:, col] = dataframe[col].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
    #remove @
    dataframe.loc[:, col] = dataframe[col].str.replace(r'@.*?(?=\s|$)'," ")
    #remove #
    dataframe.loc[:, col] = dataframe[col].str.replace(r'#.*?(?=\s|$)'," ")
    #remove special characters
    dataframe.loc[:, col] = dataframe[col].str.replace("\W"," ")
    #remove digits
    dataframe.loc[:, col] = dataframe[col].str.replace('\d+', '')
    

    return dataframe
neat_df = clean_text(neat_df, "text")
neat_df.tail(50)

,created_at,place.country,place.country_code,place.name,is_quote_status,lang,source,text,user.created_at,user.description,user.name,user.location,place.place_type,user.screen_name,in_reply_to_screen_name,geo.type,geo.coordinates,create_dtm,accounts,hashtag
index,,,,,,,,,,,,,,,,,,,,
1970-01-01 00:00:00.000557660+00:00,Sun Aug 14 23:58:26 +0000 2016,United States,US,Missouri,False,en,"<a href=""http://twitter.com/download/android"" ...",I knew exactly what means when he speaks s...,Mon Apr 11 21:52:17 +0000 2016,None,virginia irene mann,None,admin,virenemann,lauraingle,None,None,2016-08-14 23:58:26+00:00,[@lauraingle],[#trump]
1970-01-01 00:00:00.000557661+00:00,Sun Aug 14 23:58:28 +0000 2016,United States,US,Tinley Park,False,und,"<a href=""http://twitter.com/download/iphone"" r...",,Thu Oct 30 21:45:46 +0000 2008,None,Independent Voter,None,city,illinoisdan,realDonaldTrump,None,None,2016-08-14 23:58:28+00:00,[@realDonaldTrump],[]
1970-01-01 00:00:00.000557662+00:00,Sun Aug 14 23:58:28 +0000 2016,México,MX,Toluca,False,es,"<a href=""http://twitter.com/download/iphone"" r...",A ver qué tal,Fri Mar 05 22:42:17 +0000 2010,Pdta. de Fundación Colosio #Toluca Maestranda ...,Diana Velázquez,México @FColosioACTOL,city,diana_vs,None,None,None,2016-08-14 23:58:28+00:00,[],"[#ALeer, #libros, #lectores, #HillaryClinton, ..."
1970-01-01 00:00:00.000557663+00:00,Sun Aug 14 23:58:30 +0000 2016,United States,US,Spring Hill,False,en,"<a href=""http://twitter.com/download/iphone"" r...",They claim he s waiting for the debates ...,Tue Apr 09 12:32:33 +0000 2013,Crazy for Trump Supporter! #MAGA #Trump2016 #A...,Claudia Kabat,"Spring Hill, FL",city,rninfl,SusanMarkell1,None,None,2016-08-14 23:58:30+00:00,"[@SusanMarkell1, @realDonaldTrump, @HillaryCli...",[]
1970-01-01 00:00:00.000557664+00:00,Sun Aug 14 23:58:30 +0000 2016,United States,US,Homestead,False,und,"<a href=""http://twitter.com/download/iphone"" r...",,Thu Feb 16 21:20:43 +0000 2012,"Knicks, dolphins, tarheels, hurricanes.",Enrique Paradoa,Miami florida,city,E_Paradoa,realDonaldTrump,None,None,2016-08-14 23:58:30+00:00,"[@realDonaldTrump, @sidrosenberg, @HillaryClin...",[]
1970-01-01 00:00:00.000557665+00:00,Sun Aug 14 23:58:33 +0000 2016,United States,US,Orchard Mesa,True,en,"<a href=""http://twitter.com/download/android"" ...",That s true otherwise Obama would put a stop t...,Fri Jul 17 14:01:07 +0000 2015,"Tax preparer, caregiver. Republican, Trump Sup...",Barb Christianson,"Grand Junction, CO",city,barb49_c,None,None,None,2016-08-14 23:58:33+00:00,[],[]
1970-01-01 00:00:00.000557666+00:00,Sun Aug 14 23:58:35 +0000 2016,United States,US,Fillmore,True,en,"<a href=""http://twitter.com/download/iphone"" r...",AMERICA READ RT RT CNN IS CORRUPT WHY ...,Mon Dec 07 06:53:55 +0000 2015,Real Estate Investing. Believe we shouldn't vo...,Common Sense Party,Not only one,city,CommonSensePar5,realDonaldTrump,None,None,2016-08-14 23:58:35+00:00,"[@realDonaldTrump, @PaulManafort]",[]
1970-01-01 00:00:00.000557667+00:00,Sun Aug 14 23:58:37 +0000 2016,Canada,CA,Toronto,False,en,"<a href=""http://twitter.com/download/android"" ...",same asshole,Fri Apr 10 03:12:04 +0000 2009,Professional cynic but my hearts not in it. We...,Chris Thompson,"Toronto, Canada",city,itsgettinglate,realDonaldTrump,None,None,2016-08-14 23:58:37+00:00,[@realDonaldTrump],[]
1970-01-01 00:00:00.000557668+00:00,Sun Aug 14 23:58:41 +0000 2016,United States,US,Clifton,False,en,"<a href=""http://twitter.com/download/android"" ...",Yuuuge Leading to Hillary s in National ...,Sun Mar 01 12:41:58 +0000 2009,President Morristown Tea Party,Jeffrey Weingarten,Far end of the Bell Curve,city,JeffreyMW,None,None,None,2016-08-14 23:58:41+00:00,"[@realDonaldTrump, @ConstitutionNat]",[#MAGA.]


In [5]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

# Create a set of stopwords
stop = set(stopwords.words('english'))

# Create a set of punctuation words 
exclude = set(string.punctuation) 

# This is the function makeing the lemmatization
lemma = WordNetLemmatizer()

# In this function we perform the entire cleaning
def clean(tweet):
    stop_free = " ".join([i for i in tweet.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

# This is the clean corpus.
neat_df['clean_tweet'] = neat_df.text.apply(clean)
neat_df

,created_at,place.country,place.country_code,place.name,is_quote_status,lang,source,text,user.created_at,user.description,...,user.location,place.place_type,user.screen_name,in_reply_to_screen_name,geo.type,geo.coordinates,create_dtm,accounts,hashtag,clean_tweet
index,,,,,,,,,,,,,,,,,,,,,
1970-01-01 00:00:00+00:00,Fri Aug 12 10:04:00 +0000 2016,United States,US,Frontenac,False,und,"<a href=""http://twitter.com/download/iphone"" r...",,Fri Sep 02 14:54:17 +0000 2011,"Fashion photographer, love action and adventur...",...,Miami,city,DUPUY77,theblaze,None,None,2016-08-12 10:04:00+00:00,"[@theblaze, @realDonaldTrump]",[],
1970-01-01 00:00:00.000000001+00:00,Fri Aug 12 10:04:02 +0000 2016,United States,US,Baton Rouge,False,en,"<a href=""http://twitter.com/download/android"" ...",ALL IN COLLUSION TOGETHER,Thu Oct 15 00:28:04 +0000 2009,"AWSCWI Pipefitter USAF NRA I remember USA, Hun...",...,"Ogletown, DE",city,redoctapus,BarackObama,None,None,2016-08-12 10:04:02+00:00,"[@BarackObama, @FBI, @LORETTALYNCH, @realDonal...","[#NOJUSTICE, #TrumpPence]",collusion together
1970-01-01 00:00:00.000000002+00:00,Fri Aug 12 10:04:10 +0000 2016,United States,US,Frontenac,False,und,"<a href=""http://twitter.com/download/iphone"" r...",,Fri Sep 02 14:54:17 +0000 2011,"Fashion photographer, love action and adventur...",...,Miami,city,DUPUY77,theblaze,None,None,2016-08-12 10:04:10+00:00,"[@theblaze, @realDonaldTrump]",[],
1970-01-01 00:00:00.000000003+00:00,Fri Aug 12 10:04:21 +0000 2016,Australia,AU,Melbourne,False,en,"<a href=""http://twitter.com/download/iphone"" r...",he will do in one year all the things you sh...,Tue Jun 02 01:59:59 +0000 2009,The best movies are written and directed by pe...,...,Australia,city,jjorgi30,HillaryClinton,None,None,2016-08-12 10:04:21+00:00,[@HillaryClinton],[],one year thing done eight
1970-01-01 00:00:00.000000004+00:00,Fri Aug 12 10:04:30 +0000 2016,United States,US,Baltimore,False,en,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",clear deliberately throwing this race in...,Mon Aug 20 09:43:48 +0000 2012,"Retired, political wonk mother of one grandmot...",...,None,city,bspence5,None,None,None,2016-08-12 10:04:30+00:00,[],"[#CNN, #newday, #Trump, #ISIS]",clear deliberately throwing race knew destabil...
1970-01-01 00:00:00.000000005+00:00,Fri Aug 12 10:04:46 +0000 2016,United States,US,Palm Springs,True,en,"<a href=""http://twitter.com/download/iphone"" r...",you wouldn t recognize a lie if it came from...,Tue May 19 03:18:19 +0000 2009,"I am a retired psychologist in Palm Springs, C...",...,"Palm Springs, CA",city,AsaDeMatteo,realDonaldTrump,None,None,2016-08-12 10:04:46+00:00,"[@realDonaldTrump,]",[#NeverTrump],recognize lie came mouth continually
1970-01-01 00:00:00.000000006+00:00,Fri Aug 12 10:04:47 +0000 2016,United Kingdom,GB,Hammersmith,True,und,"<a href=""http://twitter.com/#!/download/ipad"" ...",,Wed Nov 21 17:21:00 +0000 2012,None,...,Stuck in the middle with you,city,JehrysTargaryen,None,None,None,2016-08-12 10:04:47+00:00,[],"[#Trump2016, #TrumpPence16, #MakeAmericaGreatA...",
1970-01-01 00:00:00.000000007+00:00,Fri Aug 12 10:04:48 +0000 2016,United States,US,Secaucus,False,en,"<a href=""http://twitter.com/download/iphone"" r...",Kid you know suing someone Thats the most ...,Sun Aug 07 00:57:29 +0000 2016,Actor/Buddhist/Eccentric/Improv Comic/Libertar...,...,"Secaucus, NJ",city,GinebraFilms,None,None,None,2016-08-12 10:04:48+00:00,"[@funnyordie, @realDonaldTrump😂💩s]",[],kid know suing someone thats beautiful thing h...
1970-01-01 00:00:00.000000008+00:00,Fri Aug 12 10:04:48 +0000 2016,United States,US,Irving,False,en,"<a href=""http://twitter.com/download/iphone"" r...",you ARE the co founder of ISIS you crooked ...,Wed Oct 28 18:34:22 +0000 2009,None,...,"Irving, TX",city,VNDISABLEDVET,HillaryClinton,None,None,2016-08-12 10:04:48+00:00,[@HillaryClinton],[],co founder isi crooked evil lying witch live


In [6]:
from sklearn.feature_extraction.text import CountVectorizer

# the vectorizer object will be used to transform text to vector form
#discard any words that appear in >80% of tweets
# words that appear in less than 100 tweets will be discarded
vectorizer = CountVectorizer(max_df=0.8, min_df=100, token_pattern='\w+|\$[\d\.]+|\S+')

# apply transformation
tf = vectorizer.fit_transform(neat_df['clean_tweet']).toarray()

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names()

In [7]:
tf_feature_names[0:20]

['absolutely',
 'account',
 'act',
 'actually',
 'ad',
 'afraid',
 'ago',
 'agree',
 'allowed',
 'already',
 'also',
 'always',
 'amazing',
 'amendment',
 'america',
 'american',
 'amp',
 'another',
 'answer',
 'anti']

In [8]:
# First we use LDA: 
from sklearn.decomposition import LatentDirichletAllocation

number_of_topics = 10

model = LatentDirichletAllocation(n_components=number_of_topics, random_state=0)

In [9]:
model.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [10]:
def display_topics(model, feature_names, no_top_words):
    topic_dict = {}
    for topic_idx, topic in enumerate(model.components_):
        topic_dict["Topic %d words" % (topic_idx)]= ['{}'.format(feature_names[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
        topic_dict["Topic %d weights" % (topic_idx)]= ['{:.1f}'.format(topic[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
    return pd.DataFrame(topic_dict)

In [11]:
no_top_words = 10
topics_df =display_topics(model, tf_feature_names, no_top_words)
topics_df

,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights,Topic 5 words,Topic 5 weights,Topic 6 words,Topic 6 weights,Topic 7 words,Topic 7 weights,Topic 8 words,Topic 8 weights,Topic 9 words,Topic 9 weights
0,medium,1501.0,one,1461.8,like,1204.3,today,849.1,get,2901.2,tax,2144.1,say,1921.5,think,675.3,hillary,2615.8,trump,4164.1
1,people,1380.4,would,1174.3,good,945.0,f,650.1,amp,2133.5,return,952.1,see,888.1,news,574.0,clinton,1603.1,u,1965.4
2,lie,1111.0,right,923.7,really,838.1,pressure,551.1,vote,1753.9,show,883.2,love,724.8,go,527.2,america,1260.3,know,1110.2
3,stop,922.3,campaign,676.3,look,728.1,wind,538.1,please,1396.1,release,700.1,u,695.7,voter,509.5,great,855.1,donald,933.4
4,never,891.6,amp,657.4,rally,622.1,rain,529.1,poll,945.6,pay,549.6,word,662.9,liar,494.1,like,602.3,obama,817.1
5,said,859.9,yes,643.1,president,610.5,ky,522.1,help,824.1,care,513.2,believe,661.9,american,461.2,amp,574.8,isi,778.0
6,even,731.6,must,607.0,thing,536.4,temp,522.1,let,588.1,u,439.5,sarcasm,600.0,press,457.1,got,498.8,supporter,762.9
7,many,690.5,new,518.1,trump,525.0,forecast,521.1,elected,520.1,email,436.9,mean,521.1,people,374.6,crooked,495.9,r,623.3
8,amp,675.6,could,518.0,oh,470.4,mph,518.1,trump,499.0,amp,384.9,hrc,462.3,big,358.7,give,481.7,mr,514.1
9,truth,613.6,trump,472.9,year,372.7,crab,518.1,job,491.4,money,372.2,idiot,385.0,n,343.1,u,449.1,make,505.0


In [12]:
#Now we use NMF: non-negative matrix factorisation
from sklearn.decomposition import NMF

number_of_topics = 10

model = NMF(n_components=number_of_topics, random_state=0, alpha=.1, l1_ratio=.5)

In [13]:
model.fit(tf)

NMF(alpha=0.1, beta_loss='frobenius', init=None, l1_ratio=0.5, max_iter=200,
    n_components=10, random_state=0, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [14]:
def display_topics(model, feature_names, no_top_words):
    topic_dict = {}
    for topic_idx, topic in enumerate(model.components_):
        topic_dict["Topic %d words" % (topic_idx)]= ['{}'.format(feature_names[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
        topic_dict["Topic %d weights" % (topic_idx)]= ['{:.1f}'.format(topic[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
    return pd.DataFrame(topic_dict)

In [15]:
no_top_words = 10
display_topics(model, tf_feature_names, no_top_words)

,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights,Topic 5 words,Topic 5 weights,Topic 6 words,Topic 6 weights,Topic 7 words,Topic 7 weights,Topic 8 words,Topic 8 weights,Topic 9 words,Topic 9 weights
0,get,10.5,trump,15.0,today,3.1,u,13.2,tax,10.0,amp,14.2,people,13.4,hillary,12.6,like,12.6,medium,8.1
1,please,6.3,donald,1.3,f,3.0,r,1.6,return,5.9,volunteer,0.5,many,2.3,clinton,3.9,look,1.6,say,6.8
2,help,3.0,vote,1.0,pressure,3.0,ur,1.1,release,2.1,area,0.4,vote,1.3,obama,1.6,would,0.7,know,5.7
3,elected,2.8,mr,0.9,wind,3.0,want,0.4,show,0.8,help,0.4,saying,1.3,crooked,1.4,sound,0.6,want,2.9
4,volunteer,2.7,supporter,0.5,rain,3.0,b,0.4,pay,0.6,elected,0.4,american,1.2,isi,0.9,make,0.4,need,2.5
5,area,2.7,go,0.5,ky,3.0,need,0.3,see,0.5,nov,0.4,think,0.5,vote,0.8,see,0.3,would,2.4
6,nov,2.7,president,0.4,temp,3.0,think,0.3,plan,0.4,please,0.4,want,0.5,foundation,0.7,need,0.3,one,2.3
7,rt,1.2,love,0.3,forecast,3.0,win,0.3,care,0.3,w,0.4,lot,0.4,email,0.6,one,0.3,lie,2.2
8,retweet,1.2,american,0.3,crab,3.0,show,0.3,hiding,0.3,dead,0.3,time,0.4,america,0.6,act,0.2,make,1.9
9,vote,0.3,rally,0.3,mph,3.0,keep,0.3,paid,0.3,r,0.3,believe,0.3,need,0.5,feel,0.2,think,1.7
